# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dunedin,-45.8742,170.5036,12.11,57,89,0.45,NZ,1706983257
1,1,manicore,-5.8092,-61.3003,30.17,75,86,1.28,BR,1706983257
2,2,nova sintra,14.8667,-24.7167,21.26,69,60,6.02,CV,1706983257
3,3,cooma,-36.2333,149.1333,14.31,63,77,0.87,AU,1706982665
4,4,ribeira grande,38.5167,-28.7000,18.22,77,20,8.05,PT,1706983258


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
125,125,mazatlan,23.2167,-106.4167,23.28,60,0,3.13,MX,1706983039
183,183,salalah,17.0151,54.0924,25.01,54,0,2.43,OM,1706983033
247,247,akot,21.0964,77.0586,23.57,31,0,2.61,IN,1706983318
285,285,dwarka,22.2394,68.9678,23.48,69,0,4.19,IN,1706982974
339,339,pacocha,-17.5833,-71.3333,25.08,65,0,3.60,PE,1706983337
353,353,al ghayzah,16.2079,52.1760,25.00,74,0,2.66,YE,1706983342
457,457,cockburn town,21.4612,-71.1419,23.15,60,0,2.17,TC,1706982827
506,506,vega de alatorre,20.0333,-96.6333,26.41,61,0,1.84,MX,1706982898


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
125,mazatlan,MX,23.2167,-106.4167,60,
183,salalah,OM,17.0151,54.0924,54,
247,akot,IN,21.0964,77.0586,31,
285,dwarka,IN,22.2394,68.9678,69,
339,pacocha,PE,-17.5833,-71.3333,65,
353,al ghayzah,YE,16.2079,52.1760,74,
457,cockburn town,TC,21.4612,-71.1419,60,
506,vega de alatorre,MX,20.0333,-96.6333,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mazatlan - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
akot - nearest hotel: No hotel found
dwarka - nearest hotel: No hotel found
pacocha - nearest hotel: No hotel found
al ghayzah - nearest hotel: No hotel found
cockburn town - nearest hotel: No hotel found
vega de alatorre - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
125,mazatlan,MX,23.2167,-106.4167,60,No hotel found
183,salalah,OM,17.0151,54.0924,54,No hotel found
247,akot,IN,21.0964,77.0586,31,No hotel found
285,dwarka,IN,22.2394,68.9678,69,No hotel found
339,pacocha,PE,-17.5833,-71.3333,65,No hotel found
353,al ghayzah,YE,16.2079,52.1760,74,No hotel found
457,cockburn town,TC,21.4612,-71.1419,60,No hotel found
506,vega de alatorre,MX,20.0333,-96.6333,61,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)